# Extração e Transformação dos Dados

## Bibliotecas Utilizadas

In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()
API_TOKEN = str(os.getenv('API_TOKEN'))

## Extração dos dados do Million Playlist Dataset

In [2]:
def read_json_file(file_path: str=None, url: str=None) -> dict:
    """
    Essa função lê um arquivo json de um caminho local ou de uma url
    e retorna um dicionário com o conteúdo do arquivo.

    :param file_path: caminho para o arquivo json local.
    :param url: caminho para o arquivo json online.

    :retorna: Um dicionário com o conteúdo do arquivo json.
    """

    if url is not None:
        response = requests.get(url)
        data = response.json()
        return data

    elif file_path is not None:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            return data
        

def extract_content_from_json(data: dict) -> pd.DataFrame:
    """
    Essa função extrai o conteúdo de um arquivo json e retorna um dataframe.

    :param data: Um dicionário com o conteúdo do arquivo json.

    :retorna: Um dataframe com o conteúdo do arquivo json.
    """
    df_musics = pd.DataFrame()

    # Itera sobre as playlists
    for playlist in data['playlists']:
        name_playlist = playlist['name']
        id_playlist = playlist['pid']

        print(f'{id_playlist} - Playlist: {name_playlist}')

        # Itera sobre as músicas da playlist
        musics_playlist = playlist['tracks']

        for music in musics_playlist:
            # name_musica = music['track_name']
            # id_musica = music['track_uri']

            # print(f'\tSong: {name_song} - ID: {song_id}')
            df_musica_aux = pd.DataFrame.from_dict(music, orient='index')
            df_musica_aux = df_musica_aux.T

            # Adiciona o nome da playlist e o ID da playlist
            df_musica_aux['playlist_name'] = name_playlist
            df_musica_aux['playlist_id'] = id_playlist

            df_musica_aux = df_musica_aux.T
            df_musics = pd.concat([df_musics, df_musica_aux], axis=1)

    return df_musics


In [3]:
# Para cada arquivo de playlist, extrai o conteúdo e salva em um arquivo csv

files_used = [
    'mpd.slice.0-999.json',
    'mpd.slice.1000-1999.json',
    'mpd.slice.2000-2999.json',
    'mpd.slice.3000-3999.json',
    'mpd.slice.4000-4999.json',
    'mpd.slice.5000-5999.json',
]

In [4]:
# for file in files_used:
#     version = file.split('.')[2].replace('-', '_')

#     data = read_json_file(file_path=f'../data/00 - Raw Data/{file}')
#     df_musics = extract_content_from_json(data)
#     df_musics.to_csv(f'../data/01 - Extracted Data/df_musics_{version}.csv', index=False)


## Transformação

### Concatenar os dados extraídos


In [5]:
df_musics_final = pd.DataFrame()

for file in files_used:
    version = file.split('.')[2].replace('-', '_')

    df = pd.read_csv(f'../data/01 - Extracted Data/df_musics_{version}.csv')

    # Keep only the columns that we need
    df = df[['track_uri', 'artist_uri', 'album_uri']]

    # Concatenate the dataframes
    df_musics_final = pd.concat([df_musics_final, df], axis=0)

# Remove duplicates
df_musics_final.drop_duplicates(inplace=True)

# Save the final dataframe
df_musics_final.to_csv('../data/01 - Extracted Data/df_musics_concatenate.csv', index=False)

df_musics_final.head(5)


,track_uri,artist_uri,album_uri
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:album:0z7pVBGOD7HCIB7S8eLkLI
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,spotify:album:25hVFAxTlDvXbx2X2QkUkE
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:album:6QPkyl04rXwTGlGlcYaRoW
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,spotify:album:6NmFmPX56pcLBOFMhIiKvF


## Limpeza dos dados

In [6]:
df_musics_final['track_uri'] = df_musics_final['track_uri'].str.replace('spotify:track:', '')
df_musics_final['artist_uri'] = df_musics_final['artist_uri'].str.replace('spotify:artist:', '')
df_musics_final['album_uri'] = df_musics_final['album_uri'].str.replace('spotify:album:', '')

df_musics_final.head(5)

,track_uri,artist_uri,album_uri
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K
1,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,0z7pVBGOD7HCIB7S8eLkLI
2,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,25hVFAxTlDvXbx2X2QkUkE
3,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,6QPkyl04rXwTGlGlcYaRoW
4,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,6NmFmPX56pcLBOFMhIiKvF


## Funções para extrair dados da API do Spotify

In [7]:
def make_req(route: str, id: str) -> requests.models.Response:
    url = f'https://api.spotify.com/v1/{route}/{id}'

    headers = {
        'Authorization': f'Bearer {API_TOKEN}'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f'Erro: {response.status_code}')
        print(f'Erro: {response.json()}')
        import sys
        sys.exit(1)

    else:
        return response


def get_track(id_track: str) -> pd.DataFrame:
    data = make_req('tracks', id_track).json()
    df = pd.DataFrame(data['tracks'])
    return df


def get_audio_features(id_track: str) -> dict:
    data = make_req('audio-features', id_track).json()
    df = pd.DataFrame(data['audio_features'])
    return df


def get_album(id_album: str) -> dict:
    data = make_req('albums', id_album).json()
    df = pd.DataFrame(data['albums'])
    return df


def get_artist_data(id_artist: str) -> dict:
    data = make_req('artists', id_artist).json()
    df = pd.DataFrame(data['artists'])
    return df

In [8]:
def get_music_features(tracks_ids: list) -> dict:

    tracks_ids_str = ','.join(tracks_ids)
    tracks_ids_str = '?ids=' + tracks_ids_str

    # 1. Get track data
    tracks_list_df = get_track(tracks_ids_str)

    # Collect album ids
    tracks_list_df['id_album'] = tracks_list_df['album'].apply(lambda x: x['id'])

    # Collect artist ids
    tracks_list_df['id_artist'] = tracks_list_df['artists'].apply(lambda x: [d['id'] for d in x])

    # Expand artist ids to separate rows
    tracks_list_df = tracks_list_df.explode('id_artist')

    
    # 2. Get audio features
    df_audio_features = pd.DataFrame()
    for i in range(0, len(tracks_ids), 50):
        tracks_ids_str = ','.join(tracks_ids[i:i+50] )
        tracks_ids_str = '?ids=' + tracks_ids_str
        df_audio_features_aux = get_audio_features(tracks_ids_str)
        df_audio_features = pd.concat([df_audio_features, df_audio_features_aux], axis=0)

    # Filtering columns
    df_audio_features = df_audio_features[['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    
    # Merge audio features with track data
    tracks_list_df = tracks_list_df.merge(df_audio_features, on='id', how='left')


    # 3. Get album data
    df_albums = pd.DataFrame()
    albums_ids = tracks_list_df['id_album'].unique().tolist()
    for i in range(0, len(albums_ids), 20):
        albums_ids_str = ','.join(albums_ids[i:i+20] )
        albums_ids_str = '?ids=' + albums_ids_str
        df_albums_aux = get_album(albums_ids_str)
        df_albums = pd.concat([df_albums, df_albums_aux], axis=0)

    # Filtering columns
    df_albums = df_albums[['id', 'name', 'popularity', 'release_date', 'total_tracks', 'type']]

    # Rename columns
    df_albums = df_albums.rename(columns={'id': 'id_album', 'name': 'album_name', 'popularity': 'album_popularity', 'release_date': 'album_release_date', 'total_tracks': 'album_total_tracks', 'type': 'album_type'})

    # Merge album data with track data
    tracks_list_df = tracks_list_df.merge(df_albums, on='id_album', how='left')
    

    # 4. Get artist data
    df_artists = pd.DataFrame()
    artists_ids = tracks_list_df['id_artist'].unique().tolist()
    for i in range(0, len(artists_ids), 50):
        artists_ids_str = ','.join(artists_ids[i:i+50] )
        artists_ids_str = '?ids=' + artists_ids_str
        df_artists_aux = get_artist_data(artists_ids_str)
        df_artists = pd.concat([df_artists, df_artists_aux], axis=0)

    # Filtering columns
    df_artists = df_artists[['id', 'name', 'genres', 'popularity', 'type', 'followers']]

    # Rename columns
    df_artists = df_artists.rename(columns={'id': 'id_artist', 'name': 'artist_name', 'genres': 'artist_genres', 'popularity': 'artist_popularity', 'type': 'artist_type', 'followers': 'artist_followers'})

    # Merge artist data with track data
    tracks_list_df = tracks_list_df.merge(df_artists, on='id_artist', how='left')

    # Collect number os followers
    tracks_list_df['artist_followers'] = tracks_list_df['artist_followers'].apply(lambda x: x['total'])

    # Expand artist ids to separate rows
    tracks_list_df = tracks_list_df.explode('artist_genres')

    # Keep only relevant artist_genres
    relevant_artist_genres = ['pop', 'hip hop', 'r&b', 'rap', 'reggae', 'rock', 'punk', 'alternative']
    tracks_list_df = tracks_list_df[tracks_list_df['artist_genres'].isin(relevant_artist_genres)]

    tracks_list_df = tracks_list_df[[
        'id', 'id_album', 'id_artist', 'name', 'explicit', 'duration_ms', 'popularity', # Track data
        'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', # Audio features
        'album_name', 'album_popularity', 'album_release_date', 'album_total_tracks', 'album_type', # Album data
        'artist_name', 'artist_genres', 'artist_popularity', 'artist_type', 'artist_followers' # Artist data
        ]]

    return tracks_list_df


In [9]:
# Para cada música (linha do df_music_final), pegar as features

tracks_ids = df_musics_final['track_uri'].unique().tolist()
len(tracks_ids)

108181

In [10]:
# Parte 1

tracks_ids_part1 = tracks_ids[:len(tracks_ids)//2]

df_musics_features_final = pd.DataFrame()
print(f'Existem {len(tracks_ids_part1)} músicas para serem processadas')

for i in range(0, len(tracks_ids_part1), 50):
    print(f'Processando músicas {i} a {i+50}')
    try:
        df_aux = get_music_features(tracks_ids_part1[i:i+50])
        df_musics_features_final = pd.concat([df_musics_features_final, df_aux], axis=0)
    except:
        print(f'Erro processando músicas {i} a {i+50}', file=open('../data/01 - Extracted Data/df_musics_features_1_error_log.txt', 'a'))
        continue

df_musics_features_final.to_csv('../data/01 - Extracted Data/df_musics_features_1.csv', index=False)

Existem 54090 músicas para serem processadas
Processando músicas 0 a 50
Processando músicas 50 a 100
Processando músicas 100 a 150
Processando músicas 150 a 200
Processando músicas 200 a 250
Processando músicas 250 a 300
Processando músicas 300 a 350
Processando músicas 350 a 400
Processando músicas 400 a 450
Processando músicas 450 a 500
Processando músicas 500 a 550
Processando músicas 550 a 600
Processando músicas 600 a 650
Processando músicas 650 a 700
Processando músicas 700 a 750
Processando músicas 750 a 800
Processando músicas 800 a 850
Processando músicas 850 a 900
Processando músicas 900 a 950
Processando músicas 950 a 1000
Processando músicas 1000 a 1050
Processando músicas 1050 a 1100
Processando músicas 1100 a 1150
Processando músicas 1150 a 1200
Processando músicas 1200 a 1250
Processando músicas 1250 a 1300
Processando músicas 1300 a 1350
Processando músicas 1350 a 1400
Processando músicas 1400 a 1450
Processando músicas 1450 a 1500
Processando músicas 1500 a 1550
Proces

In [ ]:
# Parte 2

tracks_ids_part2 = tracks_ids[len(tracks_ids)//2:]

df_musics_features_final = pd.DataFrame()
print(f'Existem {len(tracks_ids_part2)} músicas para serem processadas')

for i in range(0, len(tracks_ids_part2), 50):
    print(f'Processando músicas {i} a {i+50}')
    try:
        df_aux = get_music_features(tracks_ids_part2[i:i+50])
        df_musics_features_final = pd.concat([df_musics_features_final, df_aux], axis=0)
    except:
        print(f'Erro processando músicas {i} a {i+50}', file=open('../data/01 - Extracted Data/df_musics_features_2_error_log.txt', 'a'))
        continue

df_musics_features_final.to_csv('../data/01 - Extracted Data/df_musics_features_2.csv', index=False)

In [ ]:
# Concatenando resultados

df_musics_features_1 = pd.read_csv('../data/01 - Extracted Data/df_musics_features_1.csv')
df_musics_features_2 = pd.read_csv('../data/01 - Extracted Data/df_musics_features_2.csv')

df_musics_final = pd.concat([df_musics_features_1, df_musics_features_2], axis=0)
df_musics_final.to_csv('../data/01 - Extracted Data/df_musics_features_final.csv', index=False)

Após os dados serem extraídos e a base final ser gerada, foi necessário fazer o upload da base para o Google Drive, para que fosse possível realizar a análise exploratória e a modelagem dos dados sem depender do hardware local.

In [ ]:
# Função que salva um csv no google drive

def save_csv_to_google_drive(dataframe): 
    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    file_name = 'df_musics_features_final.csv'
    file_path = '../data/01 - Extracted Data/df_musics_features_final.csv'
    dataframe.to_csv(file_path, index=False)

    # Create & upload a file.
    uploaded = drive.CreateFile({'title': file_name})
    uploaded.SetContentFile(file_path)
    uploaded.Upload()
    print('Uploaded file with ID {}'.format(uploaded.get('id')))
